In [1]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from io import BytesIO
import requests
import numpy as np
import pandas as pd
import json

In [2]:
df = pd.read_pickle('filter_items_image_data_Clothing.pkl')
df

,category,description,title,brand,feature,rank,date,asin,image,imageURLHighRes,also_view,price,fit,also_buy,main_cat,tech1,details,similar_item,tech2
3,"[Clothing, Shoes & Jewelry, Luggage & Travel G...",[The Hottest Bag in Town! Brand: Anello Condit...,Japan Anello Backpack Unisex BLACK LARGE PU LE...,Anello,"[PU Leather, Imported, Size: Large PU Leather ...","635,761inClothing,ShoesJewelry(",5 star,0204444403,https://images-na.ssl-images-amazon.com/images...,[https://images-na.ssl-images-amazon.com/image...,"[B01DLVYOPG, B07CJF5HLR, B01MSS59Y6, B071YBC1J...",$65.99,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,"[Clothing, Shoes & Jewelry, Men, Accessories]","[Beautifully rendered, heartbreakingly adorabl...",Itty-Bitty Hats: cute and cuddly caps to knit ...,Visit Amazon's Susan B. Anderson Page,NaN,"441,908inBooks(",NaN,1579652956,nan,NaN,"[0823099032, 1612124801, 1579653766, 157965334...",$18.91,NaN,"[1579653766, 1579653340, 0823099032, 160900215...",Books,NaN,NaN,NaN,NaN
24,"[Clothing, Shoes & Jewelry, Luggage & Travel G...","[These convenient, self-adhesive name tags are...",Carson Dellosa Kids Name Tags (150003),Carson-Dellosa,"[40 self-adhesive name tags per pack, Availabl...",">#134,123 in Office Products (See top 100)>#1,...","November 23, 2010",1936023857,nan,NaN,"[B01GFTI4QQ, B008G0WGDU, 1594419019, B00D5T3PF...",$3.99,NaN,"[B00D5T3PF0, 193602392X, B002IXIWRA, B002IXEMG...",Office Products,"{'Brand Name': 'Carson-Dellosa', 'Item Weight'...",NaN,NaN,NaN
45,"[Clothing, Shoes & Jewelry, Women, Clothing, S...",NaN,satin,Veneziana,"[Imported, Package Dimensions:\n ...","847,304inClothing,ShoesJewelry(",5 star,5120053645,nan,NaN,NaN,NaN,NaN,"[B07BL2M7ZN, B00F31J2L4, B07CLL59B1, B0130Q7I5...",NaN,NaN,NaN,NaN,NaN
46,"[Clothing, Shoes & Jewelry, Women, Maternity, ...",[Top Length (Neck to Bottom Hem) Small - 29 in...,sofsy Soft-Touch Rayon Blend Tie Front Nursing...,NaN,[SAVE 10% WHEN YOU ADD 2 OR MORE sofsy PRODUCT...,"87,615inClothing,ShoesJewelry(",5 star,5120053084,https://images-na.ssl-images-amazon.com/images...,[https://images-na.ssl-images-amazon.com/image...,"[B01MCR5XPH, B07D7S6QQ8, B0787W1LYK, B07665XRC...",NaN,"class=""a-normal a-align-center a-spacing-smal...","[B01MCR5XPH, B0787W1LYK, B06W54L1MR, B07L891QX...",NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2685024,"[Clothing, Shoes & Jewelry, Women, Jewelry, Ne...",[HONGYE : Best Jewelry Choice You Could Never ...,HONGYE Fashion Three-dimensional Love Heart Ne...,HONGYE,[Material:gold plated and silver plated decora...,"528,223inClothing,ShoesJewelry(",HONGYE,B01HJGJ512,https://images-na.ssl-images-amazon.com/images...,[https://images-na.ssl-images-amazon.com/image...,"[B01N1OUR9J, B07FSPM8JH, B07C3PJM3Q, B075BJJ19...",$16.99,NaN,"[B07FSPM8JH, B01N1OUR9J, B01NCMQ5DH, B01MTS6I5...",NaN,NaN,NaN,NaN,NaN
2685027,"[Clothing, Shoes & Jewelry, Women, Clothing, S...",[For G&PL No Show & Liner Socks This is a Bra...,For G & PL Women Silicone Anti-Slip No Show Bo...,For G and PL,"[Material: 60% Cotton,37% Lace,3% Spandex,100%...","1,426,073inClothing,ShoesJewelry(",5 star,B01HJGMHBW,https://images-na.ssl-images-amazon.com/images...,[https://images-na.ssl-images-amazon.com/image...,NaN,NaN,NaN,[B07C1GPFTY],NaN,NaN,NaN,NaN,NaN
2685044,"[Clothing, Shoes & Jewelry, Novelty & More, Je...",[If You Would Like to Purchase a Mermaid Neckl...,"Best Wing Jewelry Resin ""10mm Iridescent Blue ...",Best Wing Earrings,"[Quantity: 1 Pair, Material: Copper and Resin,...","1,447,007inClothing,ShoesJewelry(",Best Wing Earrings,B01HJHM9UK,https://images-na.ssl-images-amazon.com/images...,[https://images-na.ssl-images-amazon.com/image...,"[B07HDVYQZF, B0783Q5Z7X, B07D26619T, B072C8Y2F...",$6.99,NaN,"[B07HDVYQZF, B016F2H90U, B07C8H5P5W, B07BPR58Q...",NaN,NaN,NaN,NaN,NaN
2685049,"[Clothing, Shoes & Jewelry, Baby, Baby Boys, C...","[Size Specification: 0-6M, Length(Tops)=13inch...",UBWH Baby Boys Clothing Short Sleeve T Shirts ...,UBWH,"[100% Cotton, Import

In [4]:
df['image'] = df['image'].astype(str)
df['image'] = df['image'].str.strip("[']'")

In [5]:
df_1 = df[:240000] 

In [7]:
model = models.vgg16(pretrained=True)
num_features = model.classifier[-1].in_features
model.classifier = torch.nn.Sequential(*list(model.classifier.children())[:-1])
for param in model.features.parameters():
    param.requires_grad = False

# Ensure the model is in evaluation mode
model.eval()

# Define the image preprocessing steps
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize the image to 224x224 pixels
    transforms.ToTensor(),  # Convert the image to a PyTorch tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def process_with_vgg16(url):
    try:
        response = requests.get(url)
        img = Image.open(BytesIO(response.content)).resize((224, 224))
        img = img.convert("RGB")

        img_tensor = preprocess(img)
        img_tensor = img_tensor.unsqueeze(0)  # Add batch dimension

        with torch.no_grad():
            prediction = model(img_tensor)
            # Optionally, apply a final layer or additional processing here
            # For simplicity, this example returns the raw model output
            return prediction.cpu().numpy()
    except Exception as e:
        print(f"Error fetching image: {e}")
        return np.nan  # Return a different marker for errors

j = 1

# Function to process multiple images from a single row
def process_images(urls):
    global j
    print(f"Processed {j} images")
    j += 1
    # Split URLs based on your actual data format; the example assumes a string list separated by ", "
    urls = urls.split(("', '")) # Adjust this based on your actual URLs format in the dataframe

    for url in urls:
        prediction = process_with_vgg16(url)
        if isinstance(prediction, np.ndarray):   # Found a valid image
            return prediction.squeeze()  # Return the first valid prediction
        else:
            continue
    return np.nan  # If all URLs were either not found or resulted in an error

df_1['VGG16_New'] = df_1['image'].apply(process_images)

/Users/jeongeuiju/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/jeongeuiju/Library/Python/3.9/lib/python/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Processed 1 images
Processed 2 images
Processed 3 images
Error fetching image: Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Processed 4 images
Error fetching image: cannot identify image file <_io.BytesIO object at 0x28340e0e0>
Error fetching image: cannot identify image file <_io.BytesIO object at 0x28365e2c0>
Error fetching image: cannot identify image file <_io.BytesIO object at 0x283716ef0>
Processed 5 images
Processed 6 images
Error fetching image: Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Processed 7 images
Processed 8 images
Processed 9 images
Processed 10 images
Error fetching image: Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Processed 11 images
Processed 12 images
Processed 13 images
Processed 14 images
Processed 15 images
Processed 16 images
Processed 17 images
Error fetching image: Invalid URL 'nan': No scheme supplied. Perhaps you meant http://nan?
Processed 18 images
Processed 19 images
Processed 

/var/folders/ss/fbwhhkzj2_99l0jshybs2tm80000gn/T/ipykernel_9895/1071830298.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_9['VGG16_New'] = df_9['image'].apply(process_images)


In [8]:
df_1.to_pickle('item_image_Clothing_Shoes_and_Jewelry.pkl') 

In [2]:
import pandas as pd

In [3]:
df_check = pd.read_pickle('item_image_Clothing_Shoes_and_Jewelry.pkl')
df_check

,category,tech1,description,fit,title,also_buy,tech2,brand,feature,rank,also_view,main_cat,similar_item,date,price,asin,image,imageURLHighRes,details,VGG16_New
37,"[Electronics, Accessories &amp; Supplies, Audi...",,[The CLIKR-5 UR5U-8780L remote control is desi...,,CLIKR-5 Time Warner Cable Remote Control UR5U-...,"[B06VTZK822, B00OSI6O7S]",,URC,[Instruction manual included],"[>#4,971 in Electronics &gt; Accessories &amp;...","[B06VTZK822, B00OSI6O7S, B00KUL8O0W, B01BHEQQY...",All Electronics,"class=""a-bordered a-horizontal-stripes a-spa...","January 31, 2013",,0511189877,https://images-na.ssl-images-amazon.com/images...,[https://images-na.ssl-images-amazon.com/image...,NaN,"[1.1608455, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5756546..."
61,"[Electronics, eBook Readers &amp; Accessories]","class=""a-keyvalue prodDetTable"" role=""present...",[The Dessin Cover in Noir for Nook Tablet help...,,Barnes &amp; Noble Dessin Leather Cover for No...,[],,Barnes &amp; Noble,"[Glazed leather exterior, faux suede lining, I...","[>#154,471 in Electronics (See top 100), >#3,7...",[],Portable Audio &amp; Accessories,,"August 25, 2012",,0594033926,https://images-na.ssl-images-amazon.com/images...,[https://images-na.ssl-images-amazon.com/image...,NaN,"[0.0, 0.0, 0.0, 0.8005604, 0.0, 0.0, 0.0, 0.0,..."
66,"[Electronics, eBook Readers &amp; Accessories,...","class=""a-keyvalue prodDetTable"" role=""present...",[Original Barnes &amp; Noble Nook Color or Tab...,,Barnes &amp; Noble Nook Color Tablet USB Cable...,[],,Barnes &amp; Noble,[<span>\n BUY MORE AND SAVE! Purchase 2...,[>#24 in Electronics &gt; eBook Readers &amp; ...,"[B01C31MQA0, B00AZRHYKW, B01MG3HKUX, B00940BV1...",Portable Audio &amp; Accessories,,"August 1, 2014",$6.04,0594459451,https://images-na.ssl-images-amazon.com/images...,[https://images-na.ssl-images-amazon.com/image...,NaN,"[1.2353559, 0.0, 0.34101626, 0.0, 0.0, 0.0, 0...."
72,"[Electronics, eBook Readers &amp; Accessories,...",,[#############################################...,,Barnes &amp; Noble OV/HB Universal Power Kit f...,[],,Barnes &amp; Noble,[Genuine Nook Charger + Genuine USB Charger Ca...,[>#55 in Electronics &gt; eBook Readers &amp; ...,"[059449771X, B00QZXQNNO, B014A9T98K, B00QZXQP5...",Computers,,"November 3, 2012",,0594481902,https://images-na.ssl-images-amazon.com/images...,[https://images-na.ssl-images-amazon.com/image...,NaN,"[0.0, 0.0, 0.0, 0.56168956, 0.0, 0.0, 0.0, 0.0..."
80,"[Electronics, eBook Readers & Accessories, Pow...","class=""a-keyvalue prodDetTable"" role=""present...",[Genuine Replacement Cable Specifically design...,,Barnes &amp; Noble Replacement Charging Sync C...,[],,Barnes & Noble,[Detachable charging cable specific for your N...,[>#40 in Electronics > eBook Readers & Accesso...,"[B06ZZB2W2X, B01N5F6RNV, B01CZTZZVM, B00940BV1...",Computers,,"November 13, 2014",$34.88,059449771X,https://images-na.ssl-images-amazon.com/images...,[https://images-na.ssl-images-amazon.com/image...,NaN,"[0.0, 0.0, 0.005195977, 0.0, 0.0, 0.0, 0.0, 0...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
223355,"[Electronics, Camera & Photo, Lighting & Studi...",,[This 13' <B>Impact Light Stand </B> is a blac...,,"Impact Heavy Duty Light Stand, Black - 13' (4m)",[],,Impact,"[Four Sections, 5/8"" Stud with 1/4""-20 Threade...",[>#40 in Camera & Photo > Camera & Photo Acces...,"[B003WGNPKQ, B00N3OFC2Q, B006U6NWFS, B0734PJZ7...",Camera & Photo,"class=""a-bordered a-horizontal-stripes a-spa...","March 28, 2011",$7.78,B004U5F3BC,https://images-na.ssl-images-amazon.com/images...,[https://images-na.ssl-images-amazon.com/image...,{},"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
223370,"[Electronics, Camera & Photo, Accessories]",,[The Sunpak Compact Pistol Grip QR Ball Head i...,,Sunpak 620-CPG Compact Pistol Grip Head for Tr...,[],,SUNPAK,"[Trigger control for the fastest, surest way t...","[>#92,319 in Camera & Photo > Camera & Photo A...","[B000JC5DAG, B009POF8V8, B00N7F4TQ6, B00TB09SK...",

In [1]:
import pandas as pd

In [2]:
cell = pd.read_pickle('Cell_Phones_and_Accessories.pkl')
cell

,reviewerID,asin,summary,reviewText,image,vote,overall,bert,roberta,VGG16
0,AAKKVN6IJJKO1,098949232X,Keep looking up at night.,This is one of the best calendars that you can...,https://images-na.ssl-images-amazon.com/images...,NaN,5.0,"[-0.0984451100230217, -0.00199960358440876, 0....","[-0.07951568067073822, 0.08175838738679886, -0...","[0.0, 0.0, 0.90600634, 0.0, 0.0, 0.0, 0.0, 0.5..."
1,A3N778P1L4YH9Y,7391002801,Case is cheaply made. If you aren't using an ...,Case is cheaply made. If you aren't using an a...,https://images-na.ssl-images-amazon.com/images...,2,1.0,"[-0.6160225868225098, 0.25027498602867126, -0....","[-0.10782118886709213, 0.11555437743663788, -0...","[0.0, 0.0, 0.0, 0.091673605, 0.78228605, 0.0, ..."
2,A34GGU76V8AE5T,7887421268,"My case looks used, could have been much bette...",I was very excited to find a really cute cover...,https://images-na.ssl-images-amazon.com/images...,19,2.0,"[-0.051112815737724304, -0.1525549292564392, -...","[-0.06908304989337921, 0.10518966615200043, -0...","[0.0, 0.0, 0.45191607, 0.0, 0.0, 0.0, 0.0, 0.0..."
3,A2FZ0P2S6KCJ5P,8288862993,"Fake, it's not a real Samsung product, bad qua...","I purchased this car charger ""Samsung Official...",https://images-na.ssl-images-amazon.com/images...,3,1.0,"[0.04953824356198311, -0.35632771253585815, 0....","[-0.07209423184394836, 0.13247931003570557, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,A2G6OURH28WI8R,9638762632,Give me the POOOWWWAAAA!!!,The PowerBear HTC One M9 Battery Case is absol...,https://images-na.ssl-images-amazon.com/images...,NaN,4.0,"[-0.5801865458488464, -0.5791118741035461, -0....","[-0.08062966167926788, 0.14100584387779236, -0...","[0.0, 0.0, 0.0, 0.20904544, 0.0, 0.0, 0.237338..."
...,...,...,...,...,...,...,...,...,...,...
179244,A3BMH52AZICG58,B01HJBS5C2,Protection that works,Dropped my phone many times and this case did ...,https://images-na.ssl-images-amazon.com/images...,NaN,5.0,"[-0.24131955206394196, -0.0698041170835495, -0...","[-0.08227464556694031, 0.13271795213222504, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
179245,A1DEBIPRY76UHQ,B01HJCR7KW,pieces starting falling off 2 days i started u...,pieces starting falling off 2 days i started ...,https://images-na.ssl-images-amazon.com/images...,NaN,1.0,"[-0.3720323145389557, 0.13811653852462769, 0.7...","[-0.08569245785474777, 0.05454415827989578, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0119427,..."
179246,AMUEAMKB4E33M,B01HJC7N4C,Get more out of your smartphone .......,Super Fun! The RockRoc 3d vr headset is waaaay...,https://images-na.ssl-images-amazon.com/images...,NaN,5.0,"[-0.20503820478916168, -0.5619103908538818, 0....","[-0.07647551596164703, 0.09452658146619797, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 2.2917056, 0.0, 0.24..."
179247,A204LEYV64Y3VW,B01HJF9OI2,I really hate complaining and bad reviews,I had it for a whole 8 days and for the first ...,https://images-na.ssl-images-amazon.com/images...,NaN,1.0,"[-0.11504127085208893, -0.17505033314228058, 0...","[-0.09449812769889832, 0.13758613169193268, -0...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.5095879, 0.0,..."
